In [ ]:
print(1)

In [ ]:
# idea:
# try import calmlib
# find some way to set up the api key universally
# idea: put at ~/.env
# asd

In [ ]:
from calmlib.utils.gpt_utils import query_gpt

In [ ]:
# from dotenv import load_dotenv
# from pathlib import Path

# p = Path('~/.env').expanduser()
# load_dotenv(p)

In [ ]:
query_gpt("What is the capital of France?", system="You are a helpful assistant.")

In [ ]:
from loguru import logger


def call_if_name_missing(name=None):
    # step 1: ask user "what do you want to do"
    if name is None:
        name = input("what do you want to do")

    # step 2: ask user "what is your name"
    # check if that is a good name for the project
    system_message = """
You're project name generator assistant.
You're given a description of a project from user.
If the description is actually already fits as a name, just return it.
Otherwise, generate a new name for the project.

1) if possible try to keep the original name, but if it's not good, generate a new one
2) maximum 3 words in the name
3) style: lowercase, with dashes, no spaces
example: "download all telegram messages" -> "load-telegram-messages"
"""

    from pydantic import BaseModel, Field

    class ProjectName(BaseModel):
        is_good_name: bool = Field(
            description="Whether the provided description is already a good name for the project"
        )
        name: str = Field(description="The name of the project")
        num_words: int = Field(description="The number of words in the name")

    query = f"Project description: {name}, current length: {len(name.split(' '))}"
    # structured output
    response = query_gpt(
        name,
        system=system_message,
        structured_output_schema=ProjectName,
    )

    if response.is_good_name:
        # format user response to lowercase and with dashes
        name = name.lower().replace(" ", "-")
        logger.info(f"The provided description is already a good name for the project: {name}")
        logger.debug("GPT response: ", str(response.model_dump_json()))
        return name
    else:
        logger.info(f"The provided description is not a good name for the project: {name}")
        logger.debug("GPT response: ", str(response.model_dump_json()))
        return response.name


res = call_if_name_missing(name="Download all telegram messages")

In [ ]:
res

In [ ]:
response